In [47]:
import yfinance as yf
import pandas as pd
import os
import warnings 
warnings.filterwarnings('ignore')

manu_stock_data = yf.download("MANU", start="2016-08-13", end="2024-05-19")
lindsell_train_data = yf.download("LTI.L", start="2016-08-13", end="2024-05-19")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [48]:
# merge all the premier league seasons data
df = pd.DataFrame()
for s in range(2016, 2024):
    data = pd.read_csv(f'fixtures/{s}_{str(s+1)[2:]}.csv')
    manu = data[((data['HomeTeam'] == 'Man United') | (data['AwayTeam'] == 'Man United'))]
    manu = manu[['HomeTeam', 'AwayTeam', 'FTR', 'PSCH', 'PSCD', 'PSCA', 'Date']]

    for i in range(len(manu)):
        if (manu.iloc[i]['HomeTeam'] == 'Man United' and manu.iloc[i]['FTR'] == 'H') or (manu.iloc[i]['AwayTeam'] == 'Man United' and manu.iloc[i]['FTR'] == 'A'):
            manu.iloc[i, 2] = 1
        else:
            if manu.iloc[i]['FTR'] == 'D':
                manu.iloc[i, 2] = 0
            else:
                manu.iloc[i, 2] = -1

    # add manu points: if FTR = 1, points = 3, if FTR = 0, points = 1, if FTR = -1, points = 0
    manu['Points'] = [3 if i == 1 else 1 if i == 0 else 0 for i in manu['FTR']]
    manu['Cumulative_points'] = manu['Points'].cumsum()
    manu['Season'] = f'{s}_{str(s+1)[2:]}'
    df = pd.concat([df, manu])

df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values(by='Date')
df = df.reset_index(drop=True)

df

,HomeTeam,AwayTeam,FTR,PSCH,PSCD,PSCA,Date,Points,Cumulative_points,Season
0,Bournemouth,Man United,1,5.40,3.65,1.78,2016-08-14,3,3,2016_17
1,Man United,Southampton,1,1.53,4.27,7.44,2016-08-19,3,6,2016_17
2,Hull,Man United,1,9.15,4.54,1.45,2016-08-27,3,9,2016_17
3,Man United,Man City,-1,2.42,3.10,3.51,2016-09-10,0,9,2016_17
4,Watford,Man United,-1,6.50,4.27,1.58,2016-09-18,0,9,2016_17
...,...,...,...,...,...,...,...,...,...,...
299,Man United,Burnley,0,1.50,5.29,5.72,2024-04-27,1,54,2023_24
300,Crystal Palace,Man United,-1,1.99,4.05,3.57,2024-05-06,0,54,2023_24
301,Man United,Arsenal,-1,9.75,7.05,1.26,2024-05-12,0,54,2023_24
302,Man United,Newcastle,1,2.73,3.90,2.47,2024-05-15,3,57,2023_24


In [49]:
# team positions across each season in the dataset
def build_allSeasons_team_positions():
    team_seasons_files = os.listdir('positions')
    team_seasons_files = [file for file in team_seasons_files if file != 'positions.csv']
    positions_df = pd.DataFrame()

    for file in team_seasons_files:
        season_df = pd.read_csv(f'positions/{file}')
        season_df['Season'] = file.split('.')[0]
        season_df = season_df[['Season', 'Match', 'Position']]
        positions_df = pd.concat([positions_df, season_df])

    positions_df = positions_df.reset_index(drop=True)
    # shift the position by 1
    positions_df['Position'] = positions_df['Position'].astype(int)
    positions_df.to_csv('positions/positions.csv', index=False)


build_allSeasons_team_positions()
positions_df = pd.read_csv('positions/positions.csv')

# add dates to the positions dataframes
def add_dates_to_positions(df):
    positions_df = pd.read_csv('positions/positions.csv')
    team_fixtures = df[(df['HomeTeam'] == "Man United") | (df['AwayTeam'] == "Man United")]
    team_fixtures = team_fixtures[['Date', 'Season']]
    positions_df['Date'] = team_fixtures['Date'].tolist()
    return positions_df


positions_df = add_dates_to_positions(df)
positions_df.to_csv('positions/positions.csv', index=False)
positions_df

,Season,Match,Position,Date
0,2022_23,1,13,2016-08-14
1,2022_23,2,20,2016-08-19
2,2022_23,3,14,2016-08-27
3,2022_23,4,7,2016-09-10
4,2022_23,5,5,2016-09-18
...,...,...,...,...
299,2023_24,34,6,2024-04-27
300,2023_24,35,8,2024-05-06
301,2023_24,36,8,2024-05-12
302,2023_24,37,8,2024-05-15


In [50]:
# merge the positions dataframes with the final_df
df = pd.merge(df, positions_df, on=['Season', 'Date'], how='left')
df = df[['Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'PSCH', 'PSCD', 'PSCA', 'Position', 'Points', 'Cumulative_points']]
df

,Season,Date,HomeTeam,AwayTeam,FTR,PSCH,PSCD,PSCA,Position,Points,Cumulative_points
0,2016_17,2016-08-14,Bournemouth,Man United,1,5.40,3.65,1.78,NaN,3,3
1,2016_17,2016-08-19,Man United,Southampton,1,1.53,4.27,7.44,NaN,3,6
2,2016_17,2016-08-27,Hull,Man United,1,9.15,4.54,1.45,NaN,3,9
3,2016_17,2016-09-10,Man United,Man City,-1,2.42,3.10,3.51,NaN,0,9
4,2016_17,2016-09-18,Watford,Man United,-1,6.50,4.27,1.58,NaN,0,9
...,...,...,...,...,...,...,...,...,...,...,...
299,2023_24,2024-04-27,Man United,Burnley,0,1.50,5.29,5.72,6.0,1,54
300,2023_24,2024-05-06,Crystal Palace,Man United,-1,1.99,4.05,3.57,8.0,0,54
301,2023_24,2024-05-12,Man United,Arsenal,-1,9.75,7.05,1.26,8.0,0,54
302,2023_24,2024-05-15,Man United,Newcastle,1,2.73,3.90,2.47,8.0,3,57


In [51]:
def compute_markup(odds_home, odds_draw, odds_away):
    return 1/odds_home + 1/odds_draw + 1/odds_away

df['Markup'] = df.apply(lambda x: compute_markup(x['PSCH'], x['PSCD'], x['PSCA']), axis=1)
df['Home_win_prob'] = 1/(df['Markup'] * df['PSCH'])
df['Draw_prob'] = 1/(df['Markup'] * df['PSCD'])
df['Away_win_prob'] = 1/(df['Markup']* df['PSCA'])

expected_points = []

for i in range(len(df)):
    if df.iloc[i]['HomeTeam'] == 'Man United':
        expected_points.append(df.iloc[i]['Home_win_prob'] * 3 + df.iloc[i]['Draw_prob'])
    else:
        expected_points.append(df.iloc[i]['Away_win_prob'] * 3 + df.iloc[i]['Draw_prob'])

df['Expected_points'] = expected_points
df['Unexpected_points'] = df['Points'] - df['Expected_points']
df = df[['Date', 'Position', 'Points', 'Cumulative_points', 'Unexpected_points']]
df

,Date,Position,Points,Cumulative_points,Unexpected_points
0,2016-08-14,NaN,3,3,1.080851
1,2016-08-19,NaN,3,6,0.852684
2,2016-08-27,NaN,3,9,0.753915
3,2016-09-10,NaN,0,9,-1.530561
4,2016-09-18,NaN,0,9,-2.089159
...,...,...,...,...,...
299,2024-04-27,6.0,1,54,-1.124189
300,2024-05-06,8.0,0,54,-1.056056
301,2024-05-12,8.0,0,54,-0.433055
302,2024-05-15,8.0,3,57,1.681051


In [52]:
trends_df = pd.read_csv('multiTimeline.csv', skiprows=3, header=None)
trends_df.columns = ['Date', 'Interest']
trends_df['Date'] = pd.to_datetime(trends_df['Date'])
trends_df = trends_df[trends_df['Date'] >= '2016-08-01']
trends_df = trends_df[trends_df['Date'] <= '2024-05-30']
trends_df = trends_df.reset_index(drop=True)
trends_df

,Date,Interest
0,2016-08-01,38
1,2016-09-01,31
2,2016-10-01,28
3,2016-11-01,27
4,2016-12-01,26
...,...,...
89,2024-01-01,32
90,2024-02-01,36
91,2024-03-01,38
92,2024-04-01,43


In [53]:
# outer join trends_df and df
new_df = pd.merge(df, trends_df, on='Date', how='outer')
# forward fill the missing values in the Interest column
new_df['Interest'] = new_df['Interest'].ffill()
# omit the rows with missing values in the Position column
new_df = new_df.dropna(subset=['Position'])
new_df = new_df.reset_index(drop=True)

# change to integer
new_df['Position'] = new_df['Position'].astype(int)
new_df['Interest'] = new_df['Interest'].astype(int)
new_df['Points'] = new_df['Points'].astype(int)
new_df['Cumulative_points'] = new_df['Cumulative_points'].astype(int)

new_df

,Date,Position,Points,Cumulative_points,Unexpected_points,Interest
0,2018-08-10,1,3,3,0.863471,35
1,2018-08-19,10,0,3,-1.976128,35
2,2018-08-27,13,0,3,-1.413185,35
3,2018-09-02,10,3,6,0.923110,28
4,2018-09-15,8,3,9,1.144879,28
...,...,...,...,...,...,...
71,2024-04-27,6,1,54,-1.124189,43
72,2024-05-06,8,0,54,-1.056056,43
73,2024-05-12,8,0,54,-0.433055,43
74,2024-05-15,8,3,57,1.681051,43


In [54]:
manu_stock_data_close = manu_stock_data['Close'].values.flatten()
manu_stock_data_open = manu_stock_data['Open'].values.flatten()
manu_stock_data_high = manu_stock_data['High'].values.flatten()
manu_stock_data_low = manu_stock_data['Low'].values.flatten()
manu_stock_data_date = manu_stock_data.index.tz_convert(None)
manu_df = pd.DataFrame([manu_stock_data_date,
                        manu_stock_data_open,
                        manu_stock_data_high,
                        manu_stock_data_low,
                        manu_stock_data_close]).T
manu_df = manu_df.rename(columns={0: 'Date', 1: 'ManU_Open', 2: 'ManU_High', 3: 'ManU_Low', 4: 'ManU_Close'})

lindsell_train_data_close = lindsell_train_data['Close'].values.flatten()
lindsell_train_data_open = lindsell_train_data['Open'].values.flatten()
lindsell_train_data_high = lindsell_train_data['High'].values.flatten()
lindsell_train_data_low = lindsell_train_data['Low'].values.flatten()
lindsell_train_data_date = lindsell_train_data.index.tz_convert(None)
lindsell_df = pd.DataFrame([lindsell_train_data_date,
                            lindsell_train_data_open,
                            lindsell_train_data_high,
                            lindsell_train_data_low,
                            lindsell_train_data_close]).T
lindsell_df = lindsell_df.rename(columns={0: 'Date', 1: 'Lindsell_Open', 2: 'Lindsell_High', 3: 'Lindsell_Low', 4: 'Lindsell_Close'})

# Ensure Date columns are in datetime64[ns] format
manu_df['Date'] = pd.to_datetime(manu_df['Date'])
lindsell_df['Date'] = pd.to_datetime(lindsell_df['Date'])

In [55]:
final_df = pd.merge(manu_df, new_df, on='Date', how='outer')

# forward fill the missing values in: Position, Points, Cumulative_points, Unexpected_points, Interest
final_df['Position'] = final_df['Position'].ffill()
final_df['Points'] = final_df['Points'].ffill()
final_df['Cumulative_points'] = final_df['Cumulative_points'].ffill()
final_df['Unexpected_points'] = final_df['Unexpected_points'].ffill()
final_df['Interest'] = final_df['Interest'].ffill()
final_df = final_df.dropna(subset=['ManU_Close'])
final_df = final_df.reset_index(drop=True)

In [56]:
# outer join the stock data with the new_df
# lindsell_train_data_close.reset_index(inplace=True)
final_df = pd.merge(final_df, lindsell_df, on='Date', how='outer')
# forward fill the missing values in: Position, Points, Cumulative_points, Unexpected_points, Interest
final_df['Position'] = final_df['Position'].ffill()
final_df['Points'] = final_df['Points'].ffill()
final_df['Cumulative_points'] = final_df['Cumulative_points'].ffill()
final_df['Unexpected_points'] = final_df['Unexpected_points'].ffill()
final_df['Interest'] = final_df['Interest'].ffill()
final_df['Lindsell_Close'] = final_df['Lindsell_Close'].ffill()
final_df = final_df.dropna(subset=['ManU_Close'])
final_df = final_df.reset_index(drop=True)

# final_df = final_df[['Date', 'ManU_Close', 'Lindsell_Close', 'Position', 'Points', 'Cumulative_points', 'Unexpected_points', 'Interest']]
final_df

,Date,ManU_Open,ManU_High,ManU_Low,ManU_Close,Position,Points,Cumulative_points,Unexpected_points,Interest,Lindsell_Open,Lindsell_High,Lindsell_Low,Lindsell_Close
0,2016-08-15,16.6,16.620001,16.469999,16.57,NaN,NaN,NaN,NaN,NaN,795.0,803.333313,777.0,783.5
1,2016-08-16,16.51,16.620001,16.309999,16.440001,NaN,NaN,NaN,NaN,NaN,798.950317,803.033386,787.104309,785.0
2,2016-08-17,16.5,16.5,16.25,16.26,NaN,NaN,NaN,NaN,NaN,797.0,806.25,770.25,785.0
3,2016-08-18,16.16,16.65,16.16,16.58,NaN,NaN,NaN,NaN,NaN,790.0,805.0,782.005981,805.0
4,2016-08-19,16.440001,16.49,16.26,16.35,NaN,NaN,NaN,NaN,NaN,805.0,850.0,795.496704,819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,2024-05-13,15.87,16.52,15.87,16.030001,8.0,0.0,54.0,-0.433055,43.0,808.0,808.0,786.0,788.0
1949,2024-05-14,16.07,16.280001,16.01,16.129999,8.0,0.0,54.0,-0.433055,43.0,796.0,810.0,791.599976,800.0
1950,2024-05-15,16.209999,16.879999,16.15,16.85,8.0,3.0,57.0,1.681051,43.0,796.0,812.0,796.0,806.0
1951,2024-05-16,16.790001,16.959999,16.639999,16.76,8.0,3.0,57.0,1.681051,43.0,794.0,810.29657,786.0,808.0


In [57]:
# save the data to csv
final_df.to_csv('data.csv')